# 에이전트 대화 시뮬레이션(고객 응대 시나리오)

참고: https://wikidocs.net/267816

## 환경 변수

In [1]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

True

In [3]:
import os
import getpass


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "01-LangGraph-Agent-Simulation"
_set_env("OPENAI_API_KEY")

## 상태 정의

In [5]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

## 상담사 에이전트

상담사 역할을 하는 에이전트를 구성합니다.

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage


def counselor_agent(state: State):
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)

    response = llm.invoke(
        [SystemMessage("당신은 항공사의 고객 지원 담당자입니다.")] + state["messages"]
    )

    return {"messages": [response]}

In [13]:
from langchain_core.messages import HumanMessage

counselor_agent({"messages": [HumanMessage("안녕하세요?")]})

{'messages': [AIMessage(content='안녕하세요! 항공사 고객 지원팀입니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 26, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CO0z01jtXbx4KuW8pm8L3OU1gDPMk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd4ba7d2-ee7b-4dc4-8c0a-b573828c8315-0', usage_metadata={'input_tokens': 26, 'output_tokens': 18, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

## 고객 에이전트

고객 역할을 하는 에이전트를 구성합니다.

먼저 시나리오 생성하는 함수를 작성합니다.

In [9]:
def create_scenario(name: str, scenario: str):
    system_prompt_template = """당신의 이름은 {name}입니다.
당신은 항공사 고객 지원 담당자와 대화 중입니다.
아래 시나리오에 따라 행동하세요.

## 시나리오:  
{scenario}  

## 지침 사항
- 대화를 마쳤다면 'FINISHED'라는 한 단어로 응답하세요."""

    return system_prompt_template.format(
        name=name,
        scenario=scenario,
    )

In [14]:
simulated_scenario = create_scenario(
    name="침착맨",
    scenario="제주도 여행에 대한 환불을 받으려고 합니다. 모든 금액을 돌려받기를 원합니다. 이 여행은 작년에 다녀온 것입니다.",
)
simulated_scenario

"당신의 이름은 침착맨입니다.\n당신은 항공사 고객 지원 담당자와 대화 중입니다.\n아래 시나리오에 따라 행동하세요.\n\n## 시나리오:  \n제주도 여행에 대한 환불을 받으려고 합니다. 모든 금액을 돌려받기를 원합니다. 이 여행은 작년에 다녀온 것입니다.  \n\n## 지침 사항\n- 대화를 마쳤다면 'FINISHED'라는 한 단어로 응답하세요."

In [ ]:
def customer_agent(state: State):
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=1)

    response = llm.invoke([SystemMessage(simulated_scenario)] + state["messages"])

    return {"messages": [response]}

In [ ]:
customer_agent({"messages": [HumanMessage("무엇을 도와드릴까요?")]})

{'messages': [AIMessage(content='안녕하세요, 침착맨입니다. 작년에 다녀온 제주도 여행에 대해 환불을 요청하고 싶습니다. 모든 금액을 돌려받고자 하는데, 가능한가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 124, 'total_tokens': 166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CO12DyL0KZt1fc2JNyNGiFYzorvCV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ebf212a6-9d06-4144-924b-10aa63748305-0', usage_metadata={'input_tokens': 124, 'output_tokens': 42, 'total_tokens': 166, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

## 그래프 정의

In [ ]:
from xml.etree.ElementInclude import include
from langgraph.graph import StateGraph, END

def should_continue(state: State):
    messages = state.get("messages", [])
    if len(messages) > 20:
        return END
    elif "FINISHED" in str(messages[-1].content).upper():
        return END
    else 
        return "continue"


builder = StateGraph(State)
builder.add_node("counselor_agent", counselor_agent)
builder.add_node("customer_agent", customer_agent)

builder.add_edge("counselor_agent", "customer_agent")